<h1 style="text-align:center;font-size:35px;">Portfolio Optimisation with multiple constraints</h1>

<br>

**The problem of portfolio optimisation can be treated as a minimization problem, where we have to find the weightage of stocks in a portfolio in such a way that gives minimum portfolio variance. Usually, there is only one constraint - sum of weights of all stocks must equal 1.**

**But there another version of the problem that has 2 constraints :-**

- **Sum of weights of stocks = 1**
- **Mean return = k (say)** ,i.e, there is a constraint on return from portfolio. 

**Mathematically, the optimization problem is as follows :-**

\begin{equation*}
Minimize(\vec{W} C \vec{W^T})
\end{equation*}
**with the constraints :-**
\begin{equation*}
\vec{W}.\vec{u^T}=1 , \vec{W}.\vec{m^T} = k
\end{equation*}

**The lagrangian equation is :-**

\begin{equation*}
\nabla (\vec{W} , \lambda_1 , \lambda_2) = 0
\end{equation*}

<h6 style="text-align:center;">where W is vector of portfolio weights & lambda is a scalar called lagrangian multiplier.</h6> 

**For portfolio optimization problem it translates to :-**

\begin{equation*}
\nabla (\vec{W} C \vec{W^T}) - \lambda \nabla(\vec{W} \vec{u^T} - 1) - \lambda \nabla(\vec{W} \vec{m^T} - k) = 0
\end{equation*}

<h6 style="text-align:center;">where C is covariance matrix & u is a vector of ones</h6>

## 1) Importing necessary libraries :-

In [1]:
import os
import numpy as np
import pandas as pd
from scipy import stats

# For seeing all columns
pd.set_option('max_columns', None)

## 2) Compiling data :-

The parent directory contains .csv files of historical data of all Nifty50 stocks. We need to find two vital information from this historical data :-

- **Mean return**
- **Variance & covariances of all stock & pairs of stocks respectively**

So, the **"Adj Close"** column is taken from the data of all stocks & merged together.

In [2]:
def compile_data(par_dir):
    """
    Input :- Full path of folder containing all CSV files.
    Output :- Dataframe containing Adj Close columns of all stocks.
    """
    main_df = pd.DataFrame()
    count = 0

    for fname in os.listdir(par_dir):
        count += 1
        df = pd.read_csv(os.path.join(par_dir, fname))
        df.set_index('Date', inplace=True)
        ticker = fname.split('.')[0]
    
        df.rename(columns={'Adj Close':ticker},
                 inplace=True)
        df.drop(['Open','High','Low','Close','Volume'],
                axis=1, inplace=True)
    
        if main_df.empty:
            main_df = df
        else:
            main_df = main_df.join(df, how='outer')

    return main_df

In [3]:
parent_dir = r"C:\Users\smitr\Desktop\FinPython\data\nifty50"
main_df = compile_data(parent_dir)

main_df.head()

,ADANIPORTS,ASIANPAINT,AXISBANK,BAJAJ-AUTO,BAJAJFINSV,BAJFINANCE,BHARTIARTL,BPCL,BRITANNIA,CIPLA,COALINDIA,DIVISLAB,DRREDDY,EICHERMOT,GAIL,GRASIM,HCLTECH,HDFC,HDFCBANK,HDFCLIFE,HEROMOTOCO,HINDALCO,HINDUNILVR,ICICIBANK,INDUSINDBK,INFY,IOC,ITC,JSWSTEEL,KOTAKBANK,LT,M&M,MARUTI,NESTLEIND,NTPC,ONGC,POWERGRID,RELIANCE,SBILIFE,SBIN,SHREECEM,SUNPHARMA,TATAMOTORS,TATASTEEL,TCS,TECHM,TITAN,ULTRACEMCO,UPL,WIPRO
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2007-12-03,171.309952,45.282879,119.439819,NaN,NaN,12.649566,397.733093,23.285940,67.864418,172.147842,NaN,331.858276,580.851074,14.936112,32.155685,328.154602,36.476810,391.356323,117.955643,NaN,464.248016,154.475372,160.342316,93.501007,108.323395,59.448395,19.838984,41.560017,37.195450,303.784943,699.698364,129.390671,933.271606,NaN,129.028152,69.782440,107.918396,597.245605,NaN,65.268867,1307.395874,78.082802,89.080574,577.100098,170.863800,187.284683,45.002460,943.352600,93.004326,70.455162
2007-12-04,183.371567,45.682194,119.686943,NaN,NaN,12.879490,394.677307,24.242567,68.153351,172.101379,NaN,330.809113,590.509521,15.492086,33.120384,325.906250,36.232716,391.192017,118.341614,NaN,459.604797,156.528030,160.731400,91.706421,106.331642,60.003811,21.508688,41.690125,37.985092,308.649109,708.982544,133.186996,928.844055,NaN,128.464127,69.040070,106.515488,582.512268,NaN,64.989471,1330.173950,77.589417,89.703835,597.930725,167.576431,193.240173,45.682449,943.210999,91.897003,71.328201
2007-12-05,189.489792,46.003487,119.154617,NaN,NaN,12.923176,396.227264,24.826365,70.013947,176.887146,NaN,358.588501,596.888550,16.657459,36.282059,324.721985,36.250149,391.584900,119.882050,NaN,458.979126,160.633255,160.303360,93.665649,115.294472,59.565468,22.155027,41.787716,39.643333,317.404877,723.356628,133.281357,934.672058,NaN,131.660080,71.955849,110.760223,590.428406,NaN,66.946533,1350.261230,79.385414,93.466759,593.066101,168.145996,189.143127,46.211319,935.232849,92.004822,71.414116
2007-12-06,190.556076,45.640892,117.012581,NaN,NaN,13.058066,404.922180,24.494007,69.124092,184.367783,NaN,354.450043,597.427734,16.881182,35.980583,314.856354,36.209476,402.210846,117.527931,NaN,454.994720,155.481720,161.587372,96.614876,116.063995,60.748539,21.553560,41.874493,40.947113,317.622742,720.400879,135.270081,940.455078,NaN,131.713593,71.350517,108.709801,584.811890,NaN,67.256592,1354.946777,77.163490,93.836174,581.177368,168.528732,197.001587,45.759457,925.413696,93.261276,70.884056
2007-12-07,192.286652,45.663834,118.958176,NaN,NaN,13.103669,412.714172,24.346607,68.909157,183.531418,NaN,358.538116,602.728455,18.113337,35.566036,311.847046,37.098713,417.660156,119.603867,NaN,453.940765,151.255661,162.326645,100.388573,115.837685,63.978554,21.221430,40.801075,42.375484,314.801453,714.063599,134.306015,941.042297,NaN,132.035873,71.055374,108.242119,578.228394,NaN,68.386726,1355.874634,79.037567,92.849808,570.145142,172.930664,216.585419,44.605808,924.280701,93.450165,72.044060


## 3) Data preparation :-

In [4]:
missing_fracs = main_df.isnull().mean().sort_values(ascending=False)
drop_cols = sorted(list(missing_fracs[missing_fracs > 0.25].index))
main_df.drop(labels=drop_cols, axis=1, inplace=True)
main_df = main_df[1700:]

main_df.shape

(1518, 47)

## 4) Finding optimal portfolio :-

Optimal portfolio is the one that shows minimum variance.

In [5]:
def get_geometric_mean(daily_returns):

    """
    Input :- Dataframe containing daily returns for portfolio stocks.
    Output :- Array containing mean annual return for portfolio stocks.
    Working :- To calculate mean annual return from daily return
               geometric mean is used.
    """

    returns_df = daily_returns.copy()
    returns_df += 1

    returns_df = returns_df.T
    returns_df.drop(returns_df.columns[[0]], axis=1, inplace=True)

    returns_df["Geom_return"] = np.power(stats.gmean(returns_df.iloc[:,1:], axis=1), 252)
    returns_df["Geom_return"] -= 1

    return returns_df["Geom_return"].values


In [9]:
daily_returns = main_df.pct_change(1)
mean_annual_rets = get_geometric_mean(daily_returns)
cov_matrix = daily_returns.cov().to_numpy()

# Vector of ones, shown as "u" in the equation shown above
ones_vect = np.ones(cov_matrix.shape[0], dtype=float)
# Inverse of covariance matrix
inv_cov_matrix = np.linalg.inv(cov_matrix)

ones_arr = np.expand_dims(ones_vect,
                          axis=1)
ret_arr = np.expand_dims(mean_annual_rets,
                         axis=1)

# Dummy variables required for calculating lagrangian multiplier values.
a = np.matmul(ones_arr.T, np.matmul(inv_cov_matrix,ones_arr))
b = np.matmul(ret_arr.T, np.matmul(inv_cov_matrix,ones_arr))
c = np.matmul(ones_arr.T, np.matmul(inv_cov_matrix,ret_arr))
d = np.matmul(ret_arr.T, np.matmul(inv_cov_matrix,ret_arr))

m = np.array([[a[0,0],b[0,0]],
              [c[0,0],d[0,0]]])
m_inv = np.linalg.inv(m)

# Say, investor wants to get 18% avg. annual return
reqd_ret = 0.18
arr = np.array([[1.0,reqd_ret]])

# Finding values for lagrangian multipliers
lambda1 = np.matmul(m_inv, arr.T)[0,0]
lambda2 = np.matmul(m_inv, arr.T)[1,0]

dummy_var = np.expand_dims(lambda1*ones_vect + lambda2*mean_annual_rets,
                           axis=0)

# Finding minimum variance portfolio weights
wgts = np.matmul(dummy_var, inv_cov_matrix)
print("Optimal portfolio weights for required return :-\n\n",wgts)

Optimal portfolio weights for required return :-

 [[-0.01276288  0.05467927 -0.02075136  0.00449953 -0.00171981  0.04122078
  -0.03749162  0.04020755  0.05855517  0.06115525  0.01870557  0.06620593
   0.01581072  0.0239148  -0.04133719  0.02615268  0.03062767  0.16517416
  -0.00110258 -0.06607168  0.08258061 -0.03837935 -0.05182395  0.03068837
   0.0601906   0.04829774  0.00442717  0.00872025  0.02024749 -0.01436472
  -0.00340877  0.0889127   0.03593277 -0.03117557  0.14832635  0.00036309
   0.00626289  0.02407718  0.00662263 -0.03307187 -0.00725607  0.09616978
  -0.00202048  0.02858053  0.009508   -0.01677131  0.07269199]]
